In [124]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [125]:
data=pd.read_csv("MyNewFile.csv")
type(data)
data.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31',
       'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41',
       'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51',
       'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61',
       'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'Y'],
      dtype='object')

In [69]:
data.info()
data.describe()

In [126]:
y = data["Y"].values

In [127]:
x=data.drop(["Y"],axis=1)
#x = (temp-np.min(temp))/(np.max(temp)-np.min(temp))

In [131]:
# separate features set X from the target column (class label) y, 
# and divide the data set to 80% for training, and 20% for testing:

x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, train_size=0.80, test_size=0.20, random_state=101)

In [129]:
len(y) ,len(x) 

(2000, 2000)

In [132]:
# create two objects from SVM, to create two different classifiers; 
# one with Polynomial kernel, and another one with RBF kernel:

rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(x_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=1).fit(x_train, y_train)

In [133]:
# To calculate the efficiency of the two models, 
# we’ll test the two classifiers using the test data set:

poly_pred = poly.predict(x_test)
rbf_pred = rbf.predict(x_test)

In [134]:
# calculate the accuracy and f1 scores for SVM with Polynomial kernel:

poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

Accuracy (Polynomial Kernel):  62.25
F1 (Polynomial Kernel):  47.77


In [135]:
# calculate the accuracy and f1 scores for SVM with RBF kernel:

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))

Accuracy (RBF Kernel):  62.25
F1 (RBF Kernel):  47.77


In [ ]:
from __future__ import print_function
import numpy as num
from sklearn import datasets, linear_model

from genetic_selection import GeneticSelectionCV


def main():
    data = datasets.data()

    # Some noisy data not correlated
    e = num.random.uniform(0, 0.2, size=(len(data), 30))

    x = num.hstack((data, e))
    Y = data.target

    estimators = linear_model.LogisticRegression(solver="liblinear", multi_class="ovr")

    selectors = GeneticSelectionCV(estimators,
                                  cv=6,
                                  verbose=2,
                                  scoring="accuracy",
                                  max_features=6,
                                  n_population=60,
                                  crossover_proba=0.6,
                                  mutation_proba=0.2,
                                  n_generations=50,
                                  crossover_independent_proba=0.6,
                                  mutation_independent_proba=0.06,
                                  tournament_size=4,
                                  n_gen_no_change=20,
                                  caching=True,
                                  n_jobs=-2)
    selectors = selectors.fit(x, Y)

    print(selectors.support_)


if __name__ == "__main__":
    main()


In [ ]:
import matplotlib.pyplot as plot
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Categorical, Integer, Continuous
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score



nsample = len(data.images)
x = data.images.reshape((nsample, -1))
y = data['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

_, axes = plot.subplots(nrows=1, ncols=4, figsize=(10, 3))
for axis, image, label in zip(axes, data.images, data.target):
    axis.set_axis_off()
    axis.imshow(image, cmap=plot.cm.gray_r, interpolation='nearest')
    axis.set_title('Training: %i' % label)
    param_grid = {'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
              'bootstrap': Categorical([True, False]),
              'max_depth': Integer(2, 30),
              'max_leaf_nodes': Integer(2, 35),
              'n_estimators': Integer(100, 300)}

classifier = RandomForestClassifier()


cv = StratifiedKFold(n_splits=3, shuffle=True)

# The main class from sklearn-genetic-opt
evolved_estimator = GASearchCV(estimator=classifier,
                              cv=cv,
                              scoring='accuracy',
                              param_grid=param_grid,
                              n_jobs=-1,
                              verbose=True)

evolved_estimator.fit(x_train, y_train)